
### Steps:

step 1: clean up columns by addressing abbreviations

Step 2: create dataframe that contain all public health sentences

Step 3: for each row, extract sentences that contain at least one keywords that belong to public health keyword lists for each sentences, put a new row into the dataframe, 

step 4: save the new pandas dataframe into excel file

Step 5: repeat step 2, 3, and 4 for natural disaster keywords



In [38]:
import pandas as pd
import re

In [39]:
#data = pd.read_csv(r"C:\Users\ysxia\Dropbox\Code\MutualFund\Data\2021_07\2021_07_MF_data_all_columns_year2010_2020_de_dup.csv")
data = pd.read_csv(r"C:\Users\Akshay\Desktop\GSU\GRA_MutualFUnds\Cleaned_Data.csv",encoding='latin1', low_memory=False)

In [40]:
data.shape

(109018, 10)

In [41]:
data.head()

,ï»¿accession#,fund_name,filing_date,filing_year,fund_CIK,stock_series#,crsp_obj,crsp_class,principal_risks,principal_strategies
0,0000318478-20-000025,"bny mellon appreciation fund, inc.",20200514,2020,318478,s000000008,EDYG,DE,nn an investment in the fund is not a ban...,"\n\n to pursue its goals, the fund normal..."
1,0000930413-11-003179,dreyfus appreciation fund inc,27-04-2011,2011,318478,s000000008,EDYG,DE,an investment in the fund is not a bank deposi...,"to pursue its goals, the fund normally invests..."
2,0000318478-12-000007,dreyfus appreciation fund inc,27-04-2012,2012,318478,s000000008,EDYG,DE,an investment in the fund is not a bank deposi...,"to pursue its goals, the fund normally invests..."
3,0000318478-13-000011,dreyfus appreciation fund inc,29-04-2013,2013,318478,s000000008,EDYG,DE,an investment in the fund is not a bank deposi...,"to pursue its goals, the fund normally invests..."
4,0000318478-14-000011,dreyfus appreciation fund inc,29-04-2014,2014,318478,s000000008,EDYG,DE,an investment in the fund is not a bank deposi...,"to pursue its goals, the fund normally invests..."


In [42]:
data['filing_year'].value_counts()

2021    10869
2019    10390
2020    10214
2018    10184
2017     9990
2016     9803
2015     9486
2014     9017
2013     8531
2012     7855
2011     7207
2010     5472
Name: filing_year, dtype: int64

In [43]:
data1 = data.dropna(subset=['principal_risks'])

In [44]:
data1['filing_year'].value_counts()

2020    10214
2018    10065
2019    10048
2017     9901
2016     9739
2015     9434
2014     8976
2021     8633
2013     8507
2012     7848
2011     7205
2010     5472
Name: filing_year, dtype: int64

In [45]:
data1.rename(columns={"ï»¿accession#": "accession#"}, inplace=True)
data1.head()

C:\Users\Akshay\anaconda3\lib\site-packages\pandas\core\frame.py:5034: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


,accession#,fund_name,filing_date,filing_year,fund_CIK,stock_series#,crsp_obj,crsp_class,principal_risks,principal_strategies
0,0000318478-20-000025,"bny mellon appreciation fund, inc.",20200514,2020,318478,s000000008,EDYG,DE,nn an investment in the fund is not a ban...,"\n\n to pursue its goals, the fund normal..."
1,0000930413-11-003179,dreyfus appreciation fund inc,27-04-2011,2011,318478,s000000008,EDYG,DE,an investment in the fund is not a bank deposi...,"to pursue its goals, the fund normally invests..."
2,0000318478-12-000007,dreyfus appreciation fund inc,27-04-2012,2012,318478,s000000008,EDYG,DE,an investment in the fund is not a bank deposi...,"to pursue its goals, the fund normally invests..."
3,0000318478-13-000011,dreyfus appreciation fund inc,29-04-2013,2013,318478,s000000008,EDYG,DE,an investment in the fund is not a bank deposi...,"to pursue its goals, the fund normally invests..."
4,0000318478-14-000011,dreyfus appreciation fund inc,29-04-2014,2014,318478,s000000008,EDYG,DE,an investment in the fund is not a bank deposi...,"to pursue its goals, the fund normally invests..."


### Extract Sentences

In [46]:
abbreviations = {'dr.': 'doctor', 'mr.': 'mister', 'bro.': 'brother', 'bro': 'brother', 'mrs.': 'mistress', 'ms.': 'miss', 'jr.': 'junior', 'sr.': 'senior',
                 'e.g.': 'for example', 'vs.': 'versus', 'U.S.': 'United States','etc.': 'etcetera', 'J.P.': 'Justice of the Peace',
                 'Inc.': 'Incorporated', 'LLC.': 'limited liability corporation', 'Co.': 'company', 'l.p.': 'limited partneship',
                 'ltd.': 'limited', 'Jan.': 'January', 'Feb.': 'February', 'Mar.': 'March', 'Apr.': 'April', 'i.e.': 'for example',
                 'Jun.': 'June', 'Jul.': 'July', 'Aug.': 'August', 'Oct.': 'October', 'Dec.': 'December', 'S.E.C.': 'SEC', 'Inv. Co. Act': 'Investment Company Act'}
terminators = ['.', '!', '?']
wrappers = ['"', "'", ')', ']', '}']

In [47]:
def find_all(a_str, sub):
    start = 0
    while True:
        start = a_str.find(sub, start)
        if start == -1:
            return
        yield start
        start += len(sub)

In [48]:
def find_sentence_end(paragraph):
    [possible_endings, contraction_locations] = [[], []]
    contractions = abbreviations.keys()
    sentence_terminators = terminators + [terminator + wrapper for wrapper in wrappers for terminator in terminators]
    for sentence_terminator in sentence_terminators:
        t_indices = list(find_all(paragraph, sentence_terminator))
        possible_endings.extend(([] if not len(t_indices) else [[i, len(sentence_terminator)] for i in t_indices]))
    for contraction in contractions:
        c_indices = list(find_all(paragraph, contraction))
        contraction_locations.extend(([] if not len(c_indices) else [i + len(contraction) for i in c_indices]))
    possible_endings = [pe for pe in possible_endings if pe[0] + pe[1] not in contraction_locations]
    if len(paragraph) in [pe[0] + pe[1] for pe in possible_endings]:
        max_end_start = max([pe[0] for pe in possible_endings])
        possible_endings = [pe for pe in possible_endings if pe[0] != max_end_start]
    possible_endings = [pe[0] + pe[1] for pe in possible_endings if sum(pe) > len(paragraph) or (sum(pe) < len(paragraph) and paragraph[sum(pe)] == ' ')]
    end = (-1 if not len(possible_endings) else max(possible_endings))
    return end

In [49]:
def find_sentences(paragraph):
    replacingList = [["?", "? "], ["!", "! "],[".", ". "],["(", " "], [")", " "], [",", " "], ["i. e.", "i.e"], 
                     ["e. g.", "e.g."],["U. S.", "U.S."], ["J. P.", "J.P."], ["l. p.", "l.p."], 
                     ["S. E. C.", "S.E.C."]]
    for items in replacingList:
        paragraph = paragraph.replace(items[0], items[1])
    paragraph = re.sub(' +', ' ', paragraph)
    if paragraph != "":
        if paragraph[0] == '?' or paragraph[0] == '.' or paragraph[0] == '!':
            paragraph = paragraph[1:]
    end = True
    sentences = []
    while end > -1:
        end = find_sentence_end(paragraph)
        if end > -1:
            sentences.append(paragraph[end:].strip())
            paragraph = paragraph[:end]
    sentences.append(paragraph)
    sentences.reverse()
    return sentences

### input paragraphs, then split into sentences by using the above function find_sentences()

In [50]:
data1['cleaned_principal_risks']= ""
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    para = data1['principal_risks'].iloc[row]
    sents = find_sentences(para)
    data1['cleaned_principal_risks'].iloc[row] = sents

<ipython-input-50-0f9802f0c64e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['cleaned_principal_risks']= ""
C:\Users\Akshay\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000


In [51]:
data1.shape

(106042, 11)

In [52]:
data1.head(1)

,accession#,fund_name,filing_date,filing_year,fund_CIK,stock_series#,crsp_obj,crsp_class,principal_risks,principal_strategies,cleaned_principal_risks
0,0000318478-20-000025,"bny mellon appreciation fund, inc.",20200514,2020,318478,s000000008,EDYG,DE,nn an investment in the fund is not a ban...,"\n\n to pursue its goals, the fund normal...",[ nn an investment in the fund is not a bank d...


In [57]:
type(data1['cleaned_principal_risks'].iloc[0])

list

In [58]:
data1.to_csv('2021_07_principal_risks_cleaned_non_dedup.csv')

### Public Health Keywords to Search Sentences that Contain one or multiple of these keywords

In [59]:
# Public Health keywords: 
ph_word = ['communicable diseases','health crises','pandemic','respiratory','illness','prevention','epidemic',
           'coronavirus','health crisis','pandemics','sanitation','global health crises',
           'covid','health screenings','pathogens','sars','global health crisis',
           'covid 19','hiv','preparedness','sars cov 2','epidemics','disease','influenza',
           'public health','virus','global health','diseases','mers','quarantines','h1n1','viruses']

ph_word = [ 'communicable   diseases',         
            'coronavirus',                          
            'covid','covid 19',                     
            'disease',                              
            'epidemic',                             
            'global health',                        
            'global health crisis',                 
            'health crises',                        
            'health crisis' ,                       
            'health screenings' ,                   
            'hiv' ,                                 
            'influenza', 'h1n1',                     
            'pandemic','epidemic',             
            'pathogens',                            
            'prevention',                           
            'public health',                        
            'quarantines',                          
            'respiratory illness',                  
            'sanitation',                           
            'sars', 'sars cov 2', 'mers',        
            'virus']                                


In [60]:
data1.columns

Index(['accession#', 'fund_name', 'filing_date', 'filing_year', 'fund_CIK',
       'stock_series#', 'crsp_obj', 'crsp_class', 'principal_risks',
       'principal_strategies', 'cleaned_principal_risks'],
      dtype='object')

In [61]:
import os
import pandas as pd
import glob
import nltk.data
import re
#from nltk.tokenize import tokenizer
from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.tokenize.punkt import PunktSentenceTokenizer, PunktParameters, PunktTrainer

pubic_health = pd.DataFrame()
ph_sents = []
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    sentences = data1['cleaned_principal_risks'].iloc[row]
    for i in range(len(sentences)):
        for word in ph_word:
            tokens = sentences[i].lower().split()
            if word in tokens:
                ph_sents.append([data1['accession#'].iloc[row],
                                 data1['filing_year'].iloc[row],
                                 data1['fund_name'].iloc[row],
                                 sentences[i]])
        
        
        #pubic_health = pubic_health.append({'accession#':data1['accession#'].iloc[row],
        #                                    'filing_year':data1['filing_year'].iloc[row],
        #                                    'fund_name':data1['fund_name'].iloc[row],                                                    
        #                                    'sentences':sentences[i]},ignore_index=True)
                                            #'sentences':re.sub(' +', ' '," ".join(sentences[i]).replace('\n',' '))},ignore_index=True)
pubic_health = pd.DataFrame(ph_sents, columns =['accession#',
                                                'filing_year',
                                                'fund_name', 
                                                'sentences'])               

processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000


In [62]:
pubic_health.shape

(16055, 4)

In [63]:
pubic_health.head()

,accession#,filing_year,fund_name,sentences
0,0000318478-20-000025,2020,"bny mellon appreciation fund, inc.",recent examples include pandemic risks related...
1,0000318478-20-000025,2020,"bny mellon appreciation fund, inc.",recent examples include pandemic risks related...
2,0000885409-20-000004,2020,"dreyfus basic money market fund, inc",recent examples include pandemic risks related...
3,0000885409-20-000004,2020,"dreyfus basic money market fund, inc",recent examples include pandemic risks related...
4,0000075176-20-000010,2020,bny mellon municipal bond fund,primarily in response to the covid-19 pandemic...


In [64]:
pubic_health['filing_year'].value_counts()

2020    8933
2021    5319
2017     272
2016     247
2018     243
2015     237
2019     209
2014     177
2012     126
2013     126
2011     110
2010      56
Name: filing_year, dtype: int64

In [65]:
pubic_health.to_excel("2021_07_public_health_keywords_sentences_dedup.xlsx")

In [66]:
 nd_word = [ 'cyclones',                   
             'death',                      
             'drought',                    
             'earthquakes',                
             'environmental damage',       
             'cloud',                      
             'disaster',                   
             'droughts',                   
             'earthquake',                 
             'fire',                       
             'flood',                      
             'hurricane',                  
             'hurricanes',                 
             'lightning',                  
             'natural disaster',           
             'natural disasters',          
             'seismic',                    
             'storms',                     
             'tornadoes',                  
             'tsunami',                    
             'underground',                
             'volcanoes',                  
             'windstorms']


In [67]:
nd_sents = []
for row in range(len(data1)):
    if row %5000 == 0:
        print ("processing row: ", row)
    sentences = data1['cleaned_principal_risks'].iloc[row]
    for i in range(len(sentences)):
        for word in nd_word:
            tokens = sentences[i].lower().split()
            if word in tokens:
                nd_sents.append([data1['accession#'].iloc[row],
                                 data1['filing_year'].iloc[row],
                                 data1['fund_name'].iloc[row],
                                 sentences[i]])
        
        
        #pubic_health = pubic_health.append({'accession#':data1['accession#'].iloc[row],
        #                                    'filing_year':data1['filing_year'].iloc[row],
        #                                    'fund_name':data1['fund_name'].iloc[row],                                                    
        #                                    'sentences':sentences[i]},ignore_index=True)
                                            #'sentences':re.sub(' +', ' '," ".join(sentences[i]).replace('\n',' '))},ignore_index=True)
natural_disaster = pd.DataFrame(nd_sents, columns =['accession#',
                                                'filing_year',
                                                'fund_name', 
                                                'sentences'])   

processing row:  0
processing row:  5000
processing row:  10000
processing row:  15000
processing row:  20000
processing row:  25000
processing row:  30000
processing row:  35000
processing row:  40000
processing row:  45000
processing row:  50000
processing row:  55000
processing row:  60000
processing row:  65000
processing row:  70000
processing row:  75000
processing row:  80000
processing row:  85000
processing row:  90000
processing row:  95000
processing row:  100000
processing row:  105000


In [68]:
natural_disaster.shape

(9162, 4)

In [69]:
natural_disaster['filing_year'].value_counts()

2020    2077
2018     978
2019     972
2017     894
2016     826
2015     782
2021     638
2014     591
2013     510
2012     449
2011     301
2010     144
Name: filing_year, dtype: int64

In [70]:
natural_disaster.to_excel("2021_07_natural_disaster_keywords_sentences_dedup.xlsx")

In [72]:
pubic_health_non_dedup = pd.read_excel('2021_07_public_health_keywords_sentences_dedup.xlsx')

In [73]:
pubic_health_non_dedup.head(2)

,Unnamed: 0,accession#,filing_year,fund_name,sentences
0,0,0000318478-20-000025,2020,"bny mellon appreciation fund, inc.",recent examples include pandemic risks related...
1,1,0000318478-20-000025,2020,"bny mellon appreciation fund, inc.",recent examples include pandemic risks related...


In [74]:
pubic_health_non_dedup['filing_year'].value_counts()

2020    8933
2021    5319
2017     272
2016     247
2018     243
2015     237
2019     209
2014     177
2012     126
2013     126
2011     110
2010      56
Name: filing_year, dtype: int64